# DeepWater 

## Convolutional Neural Networks

### AlexNet Demo - MXNet

In this demo you will learn how to build a simple AlexNet Model usix MXNET.

![alexnet](./images/alexnet.png)

[Krizhevsky, Alex, Ilya Sutskever, and Geoffrey E. Hinton. "Imagenet classification with deep convolutional neural networks." Advances in neural information processing systems. 2012.]()

In [7]:
from __future__ import print_function
import sys, os
import h2o
from h2o.estimators.deepwater import H2ODeepWaterEstimator
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,1 day 22 hours 13 mins
H2O cluster version:,3.11.0.99999
H2O cluster version age:,2 days
H2O cluster name:,fmilo
H2O cluster total nodes:,1
H2O cluster free memory:,8.96 Gb
H2O cluster total cores:,40
H2O cluster allowed cores:,40
H2O cluster status:,"locked, healthy"
H2O connection url:,http://localhost:54321
H2O connection proxy:,None


In [8]:
def alexnet(num_classes):
    import mxnet as mx
    input_data = mx.symbol.Variable(name="data")
    # stage 1
    conv1 = mx.symbol.Convolution(
        data=input_data, kernel=(11, 11), stride=(4, 4), num_filter=96)
    relu1 = mx.symbol.Activation(data=conv1, act_type="relu")
    pool1 = mx.symbol.Pooling(
        data=relu1, pool_type="max", kernel=(3, 3), stride=(2,2))
    lrn1 = mx.symbol.LRN(data=pool1, alpha=0.0001, beta=0.75, knorm=1, nsize=5)
    # stage 2
    conv2 = mx.symbol.Convolution(
        data=lrn1, kernel=(5, 5), pad=(2, 2), num_filter=256)
    relu2 = mx.symbol.Activation(data=conv2, act_type="relu")
    pool2 = mx.symbol.Pooling(data=relu2, kernel=(3, 3), stride=(2, 2), pool_type="max")
    lrn2 = mx.symbol.LRN(data=pool2, alpha=0.0001, beta=0.75, knorm=1, nsize=5)
    # stage 3
    conv3 = mx.symbol.Convolution(
        data=lrn2, kernel=(3, 3), pad=(1, 1), num_filter=384)
    relu3 = mx.symbol.Activation(data=conv3, act_type="relu")
    conv4 = mx.symbol.Convolution(
        data=relu3, kernel=(3, 3), pad=(1, 1), num_filter=384)
    relu4 = mx.symbol.Activation(data=conv4, act_type="relu")
    conv5 = mx.symbol.Convolution(
        data=relu4, kernel=(3, 3), pad=(1, 1), num_filter=256)
    relu5 = mx.symbol.Activation(data=conv5, act_type="relu")
    pool3 = mx.symbol.Pooling(data=relu5, kernel=(3, 3), stride=(2, 2), pool_type="max")
    # stage 4
    flatten = mx.symbol.Flatten(data=pool3)
    fc1 = mx.symbol.FullyConnected(data=flatten, num_hidden=4096)
    relu6 = mx.symbol.Activation(data=fc1, act_type="relu")
    dropout1 = mx.symbol.Dropout(data=relu6, p=0.5)
    # stage 5
    fc2 = mx.symbol.FullyConnected(data=dropout1, num_hidden=4096)
    relu7 = mx.symbol.Activation(data=fc2, act_type="relu")
    dropout2 = mx.symbol.Dropout(data=relu7, p=0.5)
    # stage 6
    fc3 = mx.symbol.FullyConnected(data=dropout2, num_hidden=num_classes)
    softmax = mx.symbol.SoftmaxOutput(data=fc3, name='softmax')
    return softmax

Test checks if Deep Water works fine with a multiclass image dataset

In [9]:
frame = h2o.import_file("../../bigdata/laptop/deepwater/imagenet/cat_dog_mouse.csv")
print(frame.head(5))
nclasses = frame[1].nlevels()[0]

Parse progress: |█████████████████████████████████████████████████████████████████████████████| 100%
C1                                                               C2
---------------------------------------------------------------  ----
bigdata/laptop/deepwater/imagenet/cat/102194502_49f003abd9.jpg   cat
bigdata/laptop/deepwater/imagenet/cat/11146807_00a5f35255.jpg    cat
bigdata/laptop/deepwater/imagenet/cat/1140846215_70e326f868.jpg  cat
bigdata/laptop/deepwater/imagenet/cat/114170569_6cbdf4bbdb.jpg   cat
bigdata/laptop/deepwater/imagenet/cat/1217664848_de4c7fc296.jpg  cat

[5 rows x 2 columns]


Creating the Alexnet model architecture from scratch using the MXNet Python API

In [10]:
mxnet_model = alexnet(nclasses)
mxnet_model.save("/tmp/symbol_alexnet-py.json")

Importing the Alexnet model architecture for training in H2O

In [11]:
model = H2ODeepWaterEstimator(
    epochs=100, 
    learning_rate=1e-2, 
    mini_batch_size=128, 
    learning_rate_annealing=1e-5,
    network='user', 
    network_definition_file="/tmp/symbol_alexnet-py.json", 
    image_shape=[224,224], 
    channels=3,
    score_interval=0, 
    train_samples_per_iteration=1000,
    gpu=True
)

In [12]:
model.train(x=[0],y=1, training_frame=frame)

deepwater Model Build progress: |█████████████████████████████████████████████████████████████| 100%


In [13]:
error = model.model_performance(train=True).mean_per_class_error()
error

0.6666666666666666